In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization

In [2]:
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Load And process data

In [11]:
def create_model(learning_rate, num_hidden_layers, num_neurons):
    model = tf.keras.models.Sequential()
    for i in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_neurons, activation='relu'))
    model.add(tf.keras.layers.Dense(2, activation='softmax'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

### Define the objective function to optimizer

In [12]:
def objective(learning_rate, num_hidden_layers, num_neurons):
    model = create_model(learning_rate, int(num_hidden_layers), int(num_neurons))
    model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=0)
    val_loss, val_acc = model.evaluate(X_test, y_test, verbose=0)
    return val_acc

### Define the search space oh hyperparameters

In [13]:
pbounds = {'learning_rate': (0.0001, 0.1),
           'num_hidden_layers': (1, 5),
           'num_neurons': (5, 50)
           }

In [14]:
optimizer = BayesianOptimization(f=objective, pbounds=pbounds, verbose=2)
optimizer.maximize(init_points=5, n_iter=20)

|   iter    |  target   | learni... | num_hi... | num_ne... |
-------------------------------------------------------------


| 1         | 0.9649    | 0.04275   | 4.48      | 7.348     |
| 2         | 0.9737    | 0.02493   | 2.949     | 46.86     |
| 3         | 0.9561    | 0.05179   | 3.177     | 9.575     |
| 4         | 0.9561    | 0.07979   | 2.007     | 33.59     |
| 5         | 0.9737    | 0.02593   | 2.763     | 48.86     |
| 6         | 0.9649    | 0.1       | 5.0       | 42.75     |
| 7         | 0.9561    | 0.0001    | 5.0       | 48.46     |
| 8         | 0.9825    | 0.04595   | 2.421     | 47.87     |
| 9         | 0.9825    | 0.07021   | 1.314     | 47.76     |
| 10        | 0.9737    | 0.09679   | 1.061     | 45.82     |
| 11        | 0.9737    | 0.09261   | 4.979     | 21.62     |
| 12        | 0.9649    | 0.06325   | 1.927     | 21.55     |
| 13        | 0.9737    | 0.1       | 1.0       | 49.39     |
| 14        | 0.9649    | 0.004215  | 5.0       | 24.7      |
| 15        | 0.9825    | 0.01354   | 5.0       | 18.84     |
| 16        | 0.9561    | 0.1       | 5.0       | 16.92     |
| 17    

In [15]:
print('Optimizer hyperparameters')
print(optimizer.max['params'])
print('Validation accuracy: {:.2f}%'.format(optimizer.max['target'] * 100))

Optimizer hyperparameters
{'learning_rate': 0.045947557907169014, 'num_hidden_layers': 2.4206495903848393, 'num_neurons': 47.867695213377104}
Validation accuracy: 98.25%
